In [11]:
!nvidia-smi


/bin/bash: line 1: nvidia-smi: command not found


In [12]:
M = 3.0
deltaT = 0.2
G = 0.5
eps = 2
e = 0.7
L = 100
BITS = 7
GRID = 1 << BITS

In [13]:
import numpy as np



class Galaxy:
    def __init__(self, N = 10):
        self.N = N
        self.vel = np.zeros((N,2), dtype=np.float32)
        self.pos = np.zeros((N,2), dtype=np.float32)
        self.masses = np.zeros(N, dtype=np.float32)
        # self.active = np.ones(N, dtype=np.float32)
        self.new = N-1
        self.morton = np.arange(N, dtype=np.int32)


    def __str__(self):
        N = self.N
        pos_min = self.pos.min(axis=0) 
        pos_max = self.pos.max(axis=0)
        vel_mag = np.linalg.norm(self.vel, axis=1)

        return (
            f"Galaxy(\n"
            f"  particles : {N}\n"
            f"  mass      : min={self.masses.min():.2f}, "
            f"max={self.masses.max():.2f}, "
            f"mean={self.masses.mean():.2f}\n"
            f"  position  : x∈[{pos_min[0]:.2f}, {pos_max[0]:.2f}], "
            f"y∈[{pos_min[1]:.2f}, {pos_max[1]:.2f}] \n"
            f"  speed     : min={vel_mag.min():.2f}, "
            f"max={vel_mag.max():.2f}, "
            f"mean={vel_mag.mean():.2f}\n"
            f")"
        )


    def add(self, mass, pos, vel):
        if(self.new<0): 
            self.new = self.N
        self.masses[self.new] = mass
        self.pos[self.new] = pos
        self.vel[self.new] = vel
        self.new -= 1


    def rando(self):

        theta = np.random.uniform(0, 2*np.pi, self.N)
        # r = np.sqrt(np.random.uniform(10**2, 50**2, self.N))
        r = np.random.uniform(10, 100, self.N)

        self.pos[:,0] = r * np.cos(theta)
        self.pos[:,1] = r * np.sin(theta)

        self.masses[:] = np.abs(np.random.normal(2, 0.1, self.N))

        for i in range(self.N):
            rvec = self.pos[i]
            dist = np.linalg.norm(rvec) + eps

            # circular orbit speed
            vmag = np.sqrt(G * M / dist)

            # 2D perpendicular (tangential direction)
            t = np.array([-rvec[1], rvec[0]]) / dist

            self.vel[i] = vmag * t



    def big_bang(self):
        # random positions in small sphere
        self.pos[:] = np.random.normal(0, 1.0, (self.N, 2))
        r = np.linalg.norm(self.pos, axis=1)
        self.pos /= r[:,None]
        self.pos *= np.random.uniform(1, 5, (self.N,1))

        # nearly uniform masses
        self.masses[:] = np.abs(np.random.normal(1.0, 0.05, self.N))

        # outward Hubble velocity
        H0 = 1.0
        self.vel[:] = H0 * self.pos





    

In [14]:
import numpy as np



def update(gx):
    N = gx.N
    acc = np.zeros((N, 2), dtype=np.float32)

    # ------------------------------
    # 1. Morton ordering
    # ------------------------------
    ix = ((gx.pos[:,0] + L) / (2*L) * (GRID-1)).astype(np.uint32)
    iy = ((gx.pos[:,1] + L) / (2*L) * (GRID-1)).astype(np.uint32)

    ix = np.clip(ix, 0, GRID-1)
    iy = np.clip(iy, 0, GRID-1)

    codes = morton2D(ix, iy)
    order = np.argsort(codes)
    gx.morton = order

    pos = gx.pos[order]
    mass = gx.masses[order]

    # ------------------------------
    # 2. Build implicit nodes (ranges)
    # ------------------------------
    nodes = [(0, N)]
    for level in range(BITS):
        new_nodes = []
        mask = 1 << (2*(BITS-1-level))

        for s, e in nodes:
            if e - s <= 1:
                new_nodes.append((s, e))
                continue

            split = s
            for i in range(s+1, e):
                if (codes[order[i]] & mask) != (codes[order[i-1]] & mask):
                    new_nodes.append((split, i))
                    split = i
            new_nodes.append((split, e))

        nodes = new_nodes

    # ------------------------------
    # 3. Compute node mass + COM
    # ------------------------------
    node_mass = []
    node_com  = []

    for s, e in nodes:
        ids = slice(s, e)
        m = mass[ids].sum()
        if m > 0:
            com = (pos[ids] * mass[ids,None]).sum(axis=0) / m
        else:
            com = np.zeros(2)
        node_mass.append(m)
        node_com.append(com)

    # ------------------------------
    # 4. Barnes–Hut force evaluation
    # ------------------------------
    theta = 0.5
    box_size = 2 * L

    for ii in range(N):
        i = order[ii]
        pi = gx.pos[i]
        ai = np.zeros(2)

        for (s,e), m, com in zip(nodes, node_mass, node_com):
            if m == 0:
                continue

            size = box_size * (e - s) / N
            d = com - pi
            dist = np.linalg.norm(d) + eps

            if (size / dist) < theta or (e - s) == 1:
                ai += G * m * d / (dist**3)

        acc[i] = ai

    # ------------------------------
    # 5. Leapfrog integration
    # ------------------------------
    gx.vel += 0.5 * acc * deltaT
    gx.pos += gx.vel * deltaT
    gx.vel += 0.5 * acc * deltaT

    # ------------------------------
    # 6. Periodic boundaries
    # ------------------------------
    gx.pos[:,0] = (gx.pos[:,0] + L) % (2*L) - L
    gx.pos[:,1] = (gx.pos[:,1] + L) % (2*L) - L


    # print(np.max(np.linalg.norm(gx.pos, axis=1)))


    # for i in range(N):
    #     for j in range(i+1,N):

    #         # if (gx.active[i] and gx.active[j]):
    #             dr = gx.pos[j]-gx.pos[i]
    #             dist = np.linalg.norm(dr)
    #             if ( dist  <  (np.sqrt(gx.masses[i])+np.sqrt(gx.masses[j]))/8  ):
    #                 collision(i,j,gx)




def morton2D(x, y):
    x = (x | (x << 8)) & 0x00FF00FF
    x = (x | (x << 4)) & 0x0F0F0F0F
    x = (x | (x << 2)) & 0x33333333
    x = (x | (x << 1)) & 0x55555555

    y = (y | (y << 8)) & 0x00FF00FF
    y = (y | (y << 4)) & 0x0F0F0F0F
    y = (y | (y << 2)) & 0x33333333
    y = (y | (y << 1)) & 0x55555555

    return x | (y << 1)




# def collision(i ,j,gx):
#     if (gx.masses[i] > gx.masses[j]): collision(j,i,gx)
#     m1 = min(gx.masses[i],gx.masses[j])
#     m2 = max(gx.masses[i],gx.masses[j])
#     if(m1*3 < m2):
#         merge(j,i,gx)
#     else:
#         inelastic_collision(i,j,gx)
    

# def inelastic_collision(i ,j,gx):
#     m1 = gx.masses[i]
#     m2 = gx.masses[j]
#     v1 = gx.vel[i]
#     v2 = gx.vel[j]
#     gx.vel[i] = (m1-e*m2)/(m1+m2)*v1 + ((1+e)*m2)/(m1+m2)*v2
#     gx.vel[j] = ((1+e)*m1)/(m1+m2)*v1 +  (m2-e*m1)/(m1+m2)*v2

# def merge(i,j,gx):
#     m1 = gx.masses[i]
#     m2 = gx.masses[j]
#     v1 = gx.vel[i]
#     v2 = gx.vel[j]

#     gx.masses[i] = m1+m2
#     gx.pos[i] = (m1 * gx.pos[i] + m2 * gx.pos[j])/(m1+m2)
#     gx.vel[i] = (m1*v1+m2*v2)/(m1+m2)
    
#     gx.masses[j] = np.random.uniform(5,10)
#     addOnMerge(j,gx)
    
#     # gx.active[j] = 0



# def addOnMerge(j,gx):
#     gx.masses[j] = np.random.uniform(5,10)
#     edge = np.random.randint(4)
#     speedx = np.random.uniform(0,2)
#     speedy = np.random.uniform(0,2)

#     if (edge == 0):
#         y = np.random.uniform(-100,100)
#         gx.pos[j] = (-100,y)
#         gx.vel[j] = (speedx, (speedy if (y<0) else -speedy))
#     elif (edge == 1):
#         x = np.random.uniform(-100,100)
#         gx.pos[j] = (x,100)
#         gx.vel[j] = ((speedx if (x<0) else -speedx), -speedy)
#     elif (edge == 2):
#         y = np.random.uniform(-100,100)
#         gx.pos[j] = (100,y)
#         gx.vel[j] = (-speedx, (speedy if (y<0) else -speedy))
#     else:
#         x = np.random.uniform(-100,100)
#         gx.pos[j] = (x,-100)
#         gx.vel[j] = ((speedx if (x<0) else -speedx), speedy)




In [15]:
import numpy as np

N = 100


def main():
    # init simulation
    gx = Galaxy(N)
    gx.rando()

    gx.add(3, (0,0), (0,0))

    # centers = [(-50,-50),(50,-50),(-50,50),(50,50)]
    # M_star = 500

    # for c in centers:
    #     gx.add(M_star, c, (0,0))

    print(gx)
  
    T = 10
    history = np.zeros((T, gx.N, 2), dtype=np.float32)

    for t in range(T):
        update(gx)
        history[t] = gx.pos
    

    # np.save("mass.npy", gx.masses)


if __name__ == "__main__":
    main()

Galaxy(
  particles : 100
  mass      : min=1.63, max=3.00, mean=2.00
  position  : x∈[-95.35, 97.92], y∈[-94.25, 95.02] 
  speed     : min=0.00, max=0.29, mean=0.17
)


In [16]:
# !zip sim_output.zip history.npy 

from google.colab import drive
drive.mount('/content/drive')
np.save("/content/drive/MyDrive/history.npy", history)


ValueError: mount failed

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>